In [1]:
!ls

3rdtry.pdf               AMPtk 2015 insects.ipynb Amy_04202017.bam


In [5]:
#process data
!amptk ion -i Amy_04202017.bam -o amy2 --full_length -f COI-F -r COI-R --cpus 5

-------------------------------------------------------
[11:00:37 AM]: OS: MacOSX 10.12.4, 24 cores, ~ 67 GB RAM. Python: 2.7.12
[11:00:38 AM]: amptk v.0.9.1, USEARCH v9.2.64, VSEARCH v2.4.3
[11:00:38 AM]: Converting Ion Torrent BAM file to FASTQ
[11:01:56 AM]: Foward primer: AGGTCAACAAATCATAAAGATATTGG,  Rev comp'd rev primer: GGATTTGGAAATTGATTAGTWCC
[11:01:56 AM]: Loading FASTQ Records
[11:02:03 AM]: 6,474,485 reads (3.6 GB)
-------------------------------------------------------
[12:24:35 PM]: Concatenating Demuxed Files
[12:25:37 PM]: 6,474,485 total reads
[12:25:37 PM]: 4,756,733 valid Barcode
[12:25:37 PM]: 4,007,883 Fwd Primer found, 2,859,265 Rev Primer found
[12:25:37 PM]: 153,351 discarded too short (< 50 bp)
[12:25:37 PM]: 2,705,914 valid output reads
[12:25:53 PM]: Found 75 barcoded samples
                Sample:  Count
                 BC.47:  68954
                 BC.16:  63988
                 BC.41:  56381
                 BC.32:  55649
                 BC.57:  53568
 

In [6]:
#drop the few samples with only 1 read
!amptk remove -i amy2.demux.fq -l BC.82 BC.94 -o amy2.clean.fq

Removed 2 samples
Kept 2705912 reads out of 2705914 total reads


In [14]:
!amptk dada2 -i amy2.clean.fq --pool --uchime_ref COI -o amy2

-------------------------------------------------------
[12:39:15 PM]: OS: MacOSX 10.12.4, 24 cores, ~ 67 GB RAM. Python: 2.7.12
[12:39:16 PM]: amptk v.0.9.3, USEARCH v9.2.64, VSEARCH v2.4.3
[12:39:16 PM]: R v3.3.1; DADA2 v1.5.0
[12:39:16 PM]: Loading FASTQ Records
[12:39:21 PM]: 2,705,912 reads (1.0 GB)
[12:39:57 PM]: Quality Filtering, expected errors < 1.0
[12:40:38 PM]: 2,120,705 reads passed
[12:40:38 PM]: Splitting FASTQ file by Sample into individual files
[12:44:58 PM]: Running DADA2 pipeline
[04:01:02 PM]: 2,020 total inferred sequences (iSeqs)
[04:01:02 PM]: 378 denovo chimeras removed
[04:01:02 PM]: 1,642 valid iSeqs
[04:01:02 PM]: Chimera Filtering (VSEARCH) using COI DB
[04:01:14 PM]: 1,534 iSeqs passed, 108 ref chimeras removed
[04:01:14 PM]: Mapping reads to DADA2 iSeqs
[04:03:59 PM]: 2,705,912 reads mapped to iSeqs (100%)
[04:03:59 PM]: Clustering iSeqs at 97% to generate biological OTUs
[04:04:00 PM]: 993 OTUs generated
[04:04:00 PM]: Mapping reads to OTUs
[04:06:03 PM

#### Just a quick note that the number of Reads mapped is wrong here in the output, apparently newer version of vsearch changed the output, but we mapped something like 95% of the reads, which is good

In [15]:
#Okay, now lets filter for index bleed
!amptk filter -i amy2.cluster.otu_table.txt -f amy2.cluster.otus.fa -b BC.13 -m CFMR_insect_mock2.fa -c in

-------------------------------------------------------
[05:29:39 PM]: OS: MacOSX 10.12.4, 24 cores, ~ 67 GB RAM. Python: 2.7.12
[05:29:39 PM]: amptk v.0.9.3, USEARCH v9.2.64, VSEARCH v2.4.3
[05:29:39 PM]: Loading OTU table: amy2.cluster.otu_table.txt
[05:29:39 PM]: OTU table contains 993 OTUs
[05:29:39 PM]: Mapping OTUs to Mock Community (USEARCH)
[05:29:39 PM]: Sorting OTU table naturally
[05:29:39 PM]: Removing OTUs according to --min_reads_otu: (OTUs with less than 2 reads from all samples)
[05:29:39 PM]: Normalizing OTU table to number of reads per sample
[05:29:39 PM]: Index bleed, samples into mock: 2.163000%.
[05:29:39 PM]: Will use value of 2.200000% for index-bleed OTU filtering.
[05:29:43 PM]: Filtering OTU table down to 971 OTUs
[05:29:43 PM]: Filtering valid OTUs
-------------------------------------------------------
OTU Table filtering finished
-------------------------------------------------------
OTU Table Stats:      amy2.cluster.stats.txt
Sorted OTU table:     amy2.

In [16]:
#I think the index bleed here is too high, looks like one of the mocks is either missing or is now a different OTU, 
#specifically looks like OTU10 and IM12 could be paralogs and perhaps DADA2 corrected IM12 to OTU12, not sure...
!amptk filter -i amy2.cluster.otu_table.txt -f amy2.cluster.otus.fa -b BC.13 -m CFMR_insect_mock2.fa -p 0.005

-------------------------------------------------------
[05:32:30 PM]: OS: MacOSX 10.12.4, 24 cores, ~ 67 GB RAM. Python: 2.7.12
[05:32:30 PM]: amptk v.0.9.3, USEARCH v9.2.64, VSEARCH v2.4.3
[05:32:30 PM]: Loading OTU table: amy2.cluster.otu_table.txt
[05:32:30 PM]: OTU table contains 993 OTUs
[05:32:30 PM]: Mapping OTUs to Mock Community (USEARCH)
[05:32:30 PM]: Sorting OTU table naturally
[05:32:30 PM]: Removing OTUs according to --min_reads_otu: (OTUs with less than 2 reads from all samples)
[05:32:30 PM]: Normalizing OTU table to number of reads per sample
[05:32:30 PM]: Index bleed, mock into samples: 81.427668%.  Index bleed, samples into mock: 2.163000%.
[05:32:30 PM]: Overwriting auto detect index-bleed, setting to 0.500000%
[05:32:34 PM]: Filtering OTU table down to 973 OTUs
[05:32:34 PM]: Filtering valid OTUs
-------------------------------------------------------
OTU Table filtering finished
-------------------------------------------------------
OTU Table Stats:      amy2.c

In [20]:
#assign taxonomy/build biom file
!amptk taxonomy -f amy2.cluster.filtered.otus.fa -i amy2.cluster.final.txt -m mapping_file.fixed.txt -d COI

-------------------------------------------------------
[05:51:59 PM]: OS: MacOSX 10.12.4, 24 cores, ~ 67 GB RAM. Python: 2.7.12
[05:52:00 PM]: amptk v.0.9.3, USEARCH v9.2.64, VSEARCH v2.4.3
[05:52:00 PM]: Loading FASTA Records
[05:52:00 PM]: 973 OTUs
[05:52:00 PM]: Global alignment OTUs with usearch_global (USEARCH)
[05:52:06 PM]: Classifying OTUs with UTAX (USEARCH)
[05:52:08 PM]: Classifying OTUs with SINTAX (USEARCH)
[05:54:03 PM]: Appending taxonomy to OTU table and OTUs
[05:54:03 PM]: Generating phylogenetic tree
[05:54:06 PM]: Taxonomy finished: amy2.cluster.taxonomy.txt
[05:54:06 PM]: Classic OTU table with taxonomy: amy2.cluster.otu_table.taxonomy.txt
[05:54:10 PM]: BIOM OTU table created: amy2.cluster.biom
[05:54:10 PM]: OTUs with taxonomy: amy2.cluster.otus.taxonomy.fa
[05:54:10 PM]: OTU phylogeny: amy2.cluster.tree.phy
-------------------------------------------------------


In [25]:
#okay, loop through some very quick stats
!Rscript --vanilla phyloseq_graphs.R amy2.cluster.biom amy2.cluster.tree.phy phyloseq

Loading required package: phyloseq
Loading required package: methods
Loading required package: ggplot2
Loading required package: plyr
Loading required package: vegan
Loading required package: permute
Loading required package: lattice
This is vegan 2.4-3
Loading required package: RColorBrewer
[1] ‘1.19.1’
[1] ‘2.2.1’
[1] ‘1.8.4’
[1] ‘2.4.3’
[1] ‘1.1.2’
[1] "LinkerPrimerSequence" "phinchID"             "date"                
[4] "ReversePrimer"        "BarcodeSequence"      "site"                
[7] "species"             
[1] "date"
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 959 taxa and 72 samples ]
sample_data() Sample Data:       [ 72 samples by 7 sample variables ]
tax_table()   Taxonomy Table:    [ 959 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 959 tips and 958 internal nodes ]
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 959 taxa and 72 samples ]
sample_data() Sample Data:       [ 72 samples by 7 sampl